<a href="https://colab.research.google.com/github/RyosukeHanaoka/TechTeacher/blob/main/optPhotoFiles%E3%81%AE%E3%82%AF%E3%83%AA%E3%83%BC%E3%83%8B%E3%83%B3%E3%82%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from PIL import Image
import os
!pip install pyheif

##以下の(1)と(2)は、使用しない方はコメントアウトすること。
# 変換する画像が含まれるディレクトリパス(右手)
input_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand'
output_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand'

# 変換する画像が含まれるディレクトリパス(左手)
#input_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand'
#output_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand'

#heicを変換する関数を定義
def heic_convert_to(heic_filename, output_filename, q=90):
    import pyheif
    from PIL import Image

    heif = pyheif.read(heic_filename)

    image = Image.frombytes(
        heif.mode,
        heif.size,
        heif.data,
        "raw",
        heif.mode,
        heif.stride,
    )

    image.save(output_filename, quality=q)

#input_dirの内部を検索
for filename in os.listdir(input_dir):
    if filename.lower().endswith('.heic'):
        input_filepath = os.path.join(input_dir, filename)
        output_filepath = os.path.join(output_dir, os.path.splitext(filename)[0] + '.jpeg')

        try:
            heic_convert_to(input_filepath, output_filepath)#関数;heic_convert_toを実行
            print(f"Converted {filename} to JPEG")#作業に成功したらこちらを表示
        except Exception as e:
            print(f"Couldn't convert {filename}: {e}")#うまくいかなければこちらを表示

In [ ]:
#ファイル名を通し番号で統一する関数"rename_files"を定義。
def rename_files(directory, prefix, start_number=1):
    counter = start_number
    for foldername, subfolders, filenames in os.walk(directory):
        for filename in filenames:
            # ファイルの絶対パスを取得
            filepath = os.path.join(foldername, filename)

            # 新しいファイル名を作成
            new_filename = f"{prefix}_{str(counter).zfill(6)}{os.path.splitext(filename)[1]}"
            new_filepath = os.path.join(foldername, new_filename)

            # ファイル名を変更
            os.rename(filepath, new_filepath)
            counter += 1

##以下の(1)と(2)は使用しない方をコメントアウトすること。
#関数"rename_files"を実行(右手用)。
rename_files('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/RA/RA_Right_Hand/', 'patients')
rename_files('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Right_Hand/', 'healthy_control')

#関数"rename_files"を実行(左手用)。
#rename_files('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/RA/RA_Left_Hand/', 'patients')
#rename_files('/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/nonRA/nonRA_Left_Hand/', 'healthy_control')

In [ ]:
!pip install -q mediapipe==0.10.0
#@title Start downloading here.
!wget -O deeplabv3.tflite -q https://storage.googleapis.com/mediapipe-models/image_segmenter/deeplab_v3/float32/1/deeplab_v3.tflite

In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# 入力および出力ディレクトリを設定
base_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/'
output_base_dir = '/content/drive/MyDrive/OptPhotoFiles/optPhotoFiles/segmented_photo/'
sub_dirs = ['RA/RA_Right_Hand', 'RA/RA_Left_Hand', 'nonRA/nonRA_Right_Hand', 'nonRA/nonRA_Left_Hand']
output_sub_dirs = ['RA/RA_Right_Hand', 'RA/RA_Left_Hand', 'nonRA/nonRA_Right_Hand', 'nonRA/nonRA_Left_Hand']

# 画像のパスを取得
image_paths = []
for sub_dir in sub_dirs:
    full_dir = os.path.join(base_dir, sub_dir)
    for filename in os.listdir(full_dir):
        if filename.endswith('.jpg'):
            image_path = os.path.join(full_dir, filename)
            image_paths.append(image_path)

# 出力ディレクトリを作成
for sub_dir in output_sub_dirs:
    os.makedirs(os.path.join(output_base_dir, sub_dir), exist_ok=True)

WHITE_COLOR = (255, 255, 255)  # white

# ImageSegmenterのオプションを作成
base_options = python.BaseOptions(model_asset_path='deeplabv3.tflite')
options = vision.ImageSegmenterOptions(base_options=base_options, output_category_mask=False)#次回はoutput_category_mask=Falseで試みる。

# ImageSegmenterを作成
with vision.ImageSegmenter.create_from_options(options) as segmenter:
    for image_path in image_paths:
        image = mp.Image.create_from_file(image_path)
        segmentation_result = segmenter.segment(image)
        category_mask = segmentation_result.category_mask

        image_data = image.numpy_view()

        # Apply the mask to set the background to white
        white_bg_image = np.zeros(image_data.shape, dtype=np.uint8)
        white_bg_image[:] = WHITE_COLOR
        condition = np.stack((category_mask.numpy_view(),) * 3, axis=-1) > 0.2
        output_image = np.where(condition, image_data, white_bg_image)

        print(f'Segmentation mask of {os.path.basename(image_path)}:')

        output_sub_dir = [d for d in output_sub_dirs if d in image_path][0]
        output_path = os.path.join(output_base_dir, output_sub_dir, os.path.basename(image_path))
        cv2.imwrite(output_path, output_image)